In [1]:
import json
import types
import requests

In [2]:
class Namespace:
    def __init__(self, adict):
        self.__dict__.update(adict)

In [51]:
base_link = "http://localhost:8080/api/v1/"
#base_link = "http://192.168.0.218:8080/api/v1/"

In [70]:
links = {
    "position":Namespace({
        "all":"position/all",
        "all_active":"position/all/active",
        "single":"position/"
    }),
    "employee":Namespace({
        "all":"employee/all",
        "single":"employee/"
    }),
    "task":Namespace({
        "all":"task/all",
        "single":"task/"
    }),
    "taskrole":Namespace({
        "all":"taskrole/all",
        "single":"taskrole/",
        "position":"taskrole/position/"#{p}
    }),
    "logbook":Namespace({
        "all":"logbook/all",
        "range":"logbook/range/", #{s}/{e}
        "prange":"logbook/range/",#{p}/{s}/{e}
        "single":"logbook/"
    }),
    "absence":Namespace({
        "all":"absence/all",
        "prange":"absence/range/",#{p}
        "single":"absence/"
    }),
    "evaluation":Namespace({
        "all":"evaluation/all",
        "single":"evaluation/",
        "position":"evaluation/position/",#{p}
    }),
    "course":Namespace({
        "all":"course/all",
        "single":"course/",
        "position":"course/position/",#{p}
    })
};links_ns = Namespace(links)

In [5]:
def get_direction_name(p):
    p = Namespace(p)
    if p.service is not None:
        subdiv = p.service["subdivision"]
        subdiv = Namespace(subdiv)
        if subdiv.direction is not None:
            return subdiv.direction["shortName"]
        else: return subdiv.shortName
    else:
        div = p.subdivision["direction"]
        if div is not None:
            return div["shortName"]
        else: return p.subdivision["shortName"]
        
def get_service_name(p):
    p = Namespace(p)
    if p.service is not None:
        return p.service["name"]
    else:
        return "AUCUN"

In [6]:
def get_employee_bio_data(p):
    emp_data_list = list()
    positions = [requests.get(base_link+links_ns.position.single+str(p)).json()]
    for p in positions:
        emp_data = dict()
        emp_data["position_id"] = p["positionId"]
        emp_data["employee_id"] = p["employee"]["employeeId"]
        emp_data["fullname"] = p["employee"]["lastName"] +" "+ p["employee"]["firstName"].strip() 
        emp_data["matricula"] = p["employee"]["matricule"]
        emp_data["rolename"] = p["role"]["name"]
        emp_data["service"] = get_service_name(p)
        emp_data["direction"] = get_direction_name(p)
        emp_data["email"] = p["employee"]["email"]
        emp_data["contact"] = p["employee"]["contact"]
        emp_data["location"] = p["location"]
        emp_data_list.append(emp_data)
    return emp_data_list[0]

In [7]:
requests.get(base_link+links_ns.position.all_active).url

'http://192.168.0.218:8080/api/v1/position/all/active'

In [8]:
get_employee_bio_data(5)

{'position_id': 5,
 'employee_id': 5,
 'fullname': 'BOKOUA ZIOGBA SEBASTIEN',
 'matricula': '1281021B',
 'rolename': 'SOUS-DIRECTEUR',
 'service': 'AUCUN',
 'direction': 'DSESP',
 'email': 'sebastien.bokoua@arti.ci',
 'contact': '',
 'location': 'ABIDJAN'}

In [9]:
ov_data = dict({
    "absence" : dict({"nb":0,"done":0}),
    "evaluation" : dict({"nb":0,"done":0}),
    "activity" : dict({"nb":0,"done":0}),
    "training" : dict({"nb":0,"done":0})
})

In [10]:
ov_data

{'absence': {'nb': 0, 'done': 0},
 'evaluation': {'nb': 0, 'done': 0},
 'activity': {'nb': 0, 'done': 0},
 'training': {'nb': 0, 'done': 0}}

In [11]:
#requests.get(base_link+links_ns.absence.prange+'5/2023-04-01/2023-04-30').json()

In [12]:
def get_absence_data(p:str, s:str, e:str):
    req = requests.get(base_link+links_ns.absence.prange+p+'/'+s+'/'+e).json()
    nb = len(req)
    done = len([1 for x in req if x["justified"]=='yes'])
    return dict({"nb":nb, "done":done})
get_absence_data("6","2023-04-01","2023-04-30")

{'nb': 3, 'done': 1}

In [26]:
emp_task = requests.get(base_link+links_ns.taskrole.position+'6').json()

In [27]:
len(emp_task)

4

In [39]:
def get_taskrole_status_list(emp_taskrole):
    return [x["task"]["status"] for x in emp_taskrole]

def computer_finished_tasks(status_list):
    return len([1 for x in status_list if x=="FINISHED"])

In [40]:
computer_finished_tasks(get_taskrole_status_list(emp_task))

1

In [50]:
def get_task_data(p:str):
    done = 0
    emp_task = requests.get(base_link+links_ns.taskrole.position+str(p)).json()
    nb = len(emp_task)
    if nb > 0:
        done = computer_finished_tasks(get_taskrole_status_list(emp_task))
    return dict({"nb":nb, "done":done})
get_task_data(6)

{'nb': 4, 'done': 1}

In [69]:
def get_evaluation_data(p:str):
    done = 0
    eval_data = requests.get(base_link+links_ns.evaluation.position+str(p)).json()
    nb = len(eval_data)
    return dict({"nb":5, "done":nb})
get_evaluation_data(10)

{'nb': 5, 'done': 1}

In [75]:
def get_course_data(p:str):
    course_data = requests.get(base_link+links_ns.course.position+str(p)).json()
    nb = len(course_data)
    done = len([1 for x in course_data if x["status"] == "FINISHED"])
    return dict({"nb":nb, "done":done})
get_course_data(6)

{'nb': 2, 'done': 1}

In [77]:
{
    "absence":get_absence_data("10","2023-04-01","2023-04-30"),
    "evaluation":get_evaluation_data(10),
    "task":get_task_data(10),
    "training":get_course_data(10)
}

{'absence': {'nb': 4, 'done': 3},
 'evaluation': {'nb': 5, 'done': 1},
 'task': {'nb': 2, 'done': 0},
 'training': {'nb': 0, 'done': 0}}